### Important Note!

This notebook ran using with a 12.2 LTS Runtime version!

There are no concerns regarding cluster size (memory and cores).

Insert `spark.databricks.delta.retentionDurationCheck.enabled false` during cluster configs creation to be able to use VACUUM properly in this notebook.

The purpose of this notebook is just to show the version of the SQL commands for Python, **always use the SQL version as a reference**, as it was the one used during the Databricks SQL course.

## Using DBFS

In [0]:
%fs ls /

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/mnt/,mnt/,0,0
dbfs:/user/,user/,0,0


In [0]:
display(dbutils.fs.ls('/'))

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/mnt/,mnt/,0,0
dbfs:/user/,user/,0,0



## Some examples of DBFS Unix-Like Commands

| Command | Result |
|--------|--------|
| `ls` | List the directory files. |
| `mkdirs` | Create a new directory. |
| `cp` | Able to copy files from one directory to another. |
| `mv` | Able to move files from one directory to another. |
| `rm` | Can remove a file from a path. |
| `rm -r` | Recursively removes nested folders or files. |
| `put` | Let you insert data or even create new data files. |
| `head` | Allows file reading in the first rows. |
| `mount` | Create a link between a workspace and cloud object storage. |
| `secrets.get` | Collect a secret from a certain scope of Databricks Secrets. |


In [0]:
%fs help

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
help(dbutils.fs)

Help on FSHandler in module dbruntime.dbutils object:

class FSHandler(builtins.object)
 |  FSHandler(fsutils, fsutils_parallel, dbcore, sc, entry_point, displayHTML)
 |  
 |  Methods defined here:
 |  
 |  __call__(self)
 |      Call self as a function.
 |  
 |  __init__(self, fsutils, fsutils_parallel, dbcore, sc, entry_point, displayHTML)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  cacheFiles = f_with_exception_handling(*args, **kwargs)
 |  
 |  cacheTable = f_with_exception_handling(*args, **kwargs)
 |  
 |  check_types(self, vars_and_types)
 |  
 |  cp = f_with_exception_handling(*args, **kwargs)
 |  
 |  create_list_from_jschema(self, jschema, create_obj_from_jschema)
 |  
 |  head = f_with_exception_handling(*args, **kwargs)
 |  
 |  help(self, method_name=None)
 |  
 |  ls = f_with_exception_handling(*args, **kwargs)
 |  
 |  mkdirs = f_with_exception_handling(*args, **kwargs)
 |  
 |  mount 

In [0]:
%fs mkdirs /databricks_example

res2: Boolean = true

In [0]:
%fs ls /databricks_example/

In [0]:
dbutils.fs.put("/databricks_example/data.csv", """Name, Age, Job
Angelo, 25, Nurse
Maria, 23, Architect
Ronaldo, 33, Actor
Jessica, 18, Student
Mara, 27, Doctor""",
True)

Wrote 110 bytes.
Out[3]: True

In [0]:
%fs ls /databricks_example

path,name,size,modificationTime
dbfs:/databricks_example/data.csv,data.csv,110,1716034036000


In [0]:
%fs head /databricks_example/data.csv

Name, Age, Job
Angelo, 25, Nurse
Maria, 23, Architect
Ronaldo, 33, Actor
Jessica, 18, Student
Mara, 27, Doctor

In [0]:
%fs ls /

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1694631178000
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/databricks_cc_example/,databricks_cc_example/,0,1694649911000
dbfs:/mnt/,mnt/,0,1694530632000
dbfs:/user/,user/,0,1693629492000
dbfs:/volume/,volume/,0,0
dbfs:/volumes/,volumes/,0,0


In [0]:
%fs ls /mnt/

In [0]:
%fs mounts

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key")
encoded_secret_key = secret_key.replace("/", "%2F")
aws_bucket_name = "databricks-eng"
mount_name = "s3_dbfs"

dbutils.fs.mount(f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1371154657537257>:6
      3 aws_bucket_name = "public-gov-data-db"
      4 mount_name = "s3_dbfs"
----> 6 dbutils.fs.mount(f"s3a://{access_key}:{secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o398.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/s3_dbfs; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/s3_dbfs
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.da

In [0]:
%fs mounts

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/s3_dbfs,s3a://public-gov-data-db,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/databricks-results,databricks-results,sse-s3
/databricks/mlflow-registry,databricks/mlflow-registry,sse-s3
/,DatabricksRoot,sse-s3


In [0]:
%fs ls /mnt/s3_dbfs/

path,name,size,modificationTime
dbfs:/mnt/s3_dbfs/remuneracao202311.csv,remuneracao202311.csv,89297725,1716016122000


In [0]:
%sql
CREATE TABLE IF NOT EXISTS remuneracao_
USING csv
OPTIONS (header "true",
      delimiter ';',
      encoding 'ISO-8859-1',
      inferSchema "true",
      path '/mnt/s3_dbfs/remuneracao202311.csv');

In [0]:
%sql
DESCRIBE remuneracao_

col_name,data_type,comment
NOME,string,null
CPF,string,null
ÓRGÃO,string,null
CARGO,string,null
FUNÇÃO,string,null
SITUAÇÃO,string,null
MÊS,int,null
ANO,int,null
CÓDIGO DO ÓRGÃO,int,null
MATRÍCULA,string,null


In [0]:
%sql
DESCRIBE DETAIL remuneracao_

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
csv,null,spark_catalog.default.remuneracao_,,dbfs:/mnt/s3_dbfs/remuneracao202311.csv,2024-05-18T12:35:04.000+0000,null,List(),null,null,Map(),null,null,null,Map()


In [0]:
display(spark.sql("DESCRIBE TABLE EXTENDED remuneracao_"))

col_name,data_type,comment
NOME,string,null
CPF,string,null
ÓRGÃO,string,null
CARGO,string,null
FUNÇÃO,string,null
SITUAÇÃO,string,null
MÊS,int,null
ANO,int,null
CÓDIGO DO ÓRGÃO,int,null
MATRÍCULA,string,null


In [0]:
#dbutils.fs.unmount(/mnt/s3_dbfs)
#%fs unmount /mnt/s3_dbfs

UsageError: Line magic function `%fs` not found.


In [0]:
%sql
SELECT * FROM remuneracao_ LIMIT 5

NOME,CPF,ÓRGÃO,CARGO,FUNÇÃO,SITUAÇÃO,MÊS,ANO,CÓDIGO DO ÓRGÃO,MATRÍCULA,REMUNERAÇÃO BÁSICA,BENEFÍCIOS,VALOR DA FUNÇÃO,COMISSÃO CONSELHEIRO,HORA EXTRA,VERBAS EVENTUAIS,VERBAS JUDICIAIS,DESCONTOS A MAIOR,LICENÇA PRÊMIO,IRRF,SEG. SOCIAL,TETO REDUTOR,OUTROS RECEBIMENTOS,OUTROS DESCONTOS OBRIGATÓRIOS,PAGAMENTOS A MAIOR,BRUTO,LÍQUIDO
DANIELA PERDIGAO MENESES LIMA,***766711**,SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO E HABITACAO DO DF,ANALISTA PLANEJ. URB. INFRA.,,ATIVO,11,2023,100004,01569163,"12831,88","640,00",",00",",00",",00",",00",",00",",00",",00","2047,66","1788,62",",00",",00",",00",",00","13471,88","9635,60"
ELIETE FERREIRA DA SILVA GOES,***692721**,SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO E HABITACAO DO DF,ANALISTA POL PUBL E GEST GOV,ASSESSOR ESPECIAL,ATIVO,11,2023,100004,01569228,"9623,69","407,27","3915,00",",00",",00",",00",",00",",00",",00","2419,84","1331,63",",00",",00",",00",",00","13945,96","10194,49"
PAULA ANDERSON DE MATOS EUSTAQUIO,***681091**,SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO E HABITACAO DO DF,ANALISTA PLANEJ. URB. INFRA.,,ATIVO,11,2023,100004,01569287,"18748,95","640,00",",00",",00",",00","18748,95",",00",",00",",00","7098,32","5249,70",",00",",00",",00",",00","38137,90","25789,88"
CARLOS RENATO COLEN DE MELO,***697806**,SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO E HABITACAO DO DF,ANALISTA PLANEJ. URB. INFRA.,ASSESSOR,ATIVO,11,2023,100004,01569295,"17429,21","640,00","2940,00",",00",",00",",00",",00",",00",",00","4049,86","2424,40",",00",",00",",00",",00","21009,21","14534,95"
ROBERTA RIBEIRO,***831606**,SECRETARIA DE ESTADO DE DESENVOLVIMENTO URBANO E HABITACAO DO DF,ANALISTA PLANEJ. URB. INFRA.,,ATIVO,11,2023,100004,0156935X,"17611,02","640,00",",00",",00",",00",",00",",00",",00",",00","3284,35","2449,86",",00",",00",",00",",00","18251,02","12516,81"


In [0]:
#FONTE DO DICIONÁRIO DE DADOS: http://dados.df.gov.br/dataset/462126f8-8a61-4cec-91a2-38615b7f70f6/resource/0514402e-cd29-440b-83e5-f97787dd1ad3/download/dicdadosremuneracaodosservidores.html
spark.sql(
"CREATE TABLE IF NOT EXISTS remuneracao_geral (nome STRING COMMENT 'NOME COMPLETO DO SERVIDOR', " + \
"                                            cpf STRING COMMENT 'Nº DO CADASTRO DE PESSOA FÍSICA MASCARADO'," + \
"                                            orgao STRING COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR'," + \
"                                            cargo STRING COMMENT 'POSIÇÃO QUE O SERVIDOR OCUPA NO ÓRGÃO DE FORMA PERMANENTE'," + \
"                                            funcao STRING COMMENT 'DESIGNAÇÃO TEMPORÁRIA PARA DESEMPENHO DE DETERMINADAS ATRIBUIÇÕES (CARGO COMISSIONADO)'," + \
"                                            situacao STRING COMMENT 'CONDIÇÃO DO SERVIDOR EM RELAÇÃO AO EXERCÍCIO DE SUAS ATIVIDADES'," + \
"                                            mes INTEGER COMMENT 'MÊS DE REFERÊNCIA'," + \
"                                            ano INTEGER COMMENT 'ANO DE REFERÊNCIA'," + \
"                                            codigo_do_orgao INTEGER COMMENT 'ÓRGÃO VINCULADO AO SERVIDOR'," + \
"                                            matricula STRING COMMENT 'MATRÍCULA DO SERVIDOR NO ÓRGÃO VINCULADO'," + \
"                                            remuneracao_basica STRING COMMENT 'REMUNERAÇÃO BÁSICA'," + \
"                                            beneficios STRING COMMENT 'VALOR DOS BENEFÍCIOS'," + \
"                                            valor_das_funcoes STRING COMMENT 'VALOR DAS FUNÇÕES'," + \
"                                            comissao_conselheiro STRING COMMENT 'COMISSÃO CONSELHEIRO'," + \
"                                            hora_extra STRING COMMENT 'TOTAL DE HORA EXTRA'," + \
"                                            verbas_eventuais STRING COMMENT 'VALOR DAS VERBAS EVENTUAIS'," + \
"                                            verbas_judiciais STRING COMMENT 'VALOR DAS VERBAS JUDICIAIS'," + \
"                                            descontos_a_maior STRING COMMENT 'VALOR DA REPOSIÇÃO DE DESCONTOS A MAIOR'," + \
"                                            licenca_premio STRING COMMENT 'VALOR DA LICENÇA PRÊMIO'," + \
"                                            irrf STRING COMMENT 'VALOR DO DESCONTO DE IMPOSTO DE RENDA RETIDO NA FONTE'," + \
"                                            seguridade_social STRING COMMENT 'VALOR DO DESCONTO DE SEGURIDADE SOCIAL'," + \
"                                            teto_redutor STRING COMMENT 'VALOR DO DESCONTO DE TETO REDUTOR'," + \
"                                            outros_recebimentos STRING COMMENT 'VALOR DE OUTROS RECEBIMENTOS'," + \
"                                            outros_descontos_obrigatorios STRING COMMENT 'VALOR DE OUTROS DESCONTOS OBRIGATÓRIOS'," + \
"                                            pagamento_a_maior STRING COMMENT 'VALOR DOS DESCONTOS DE PAGAMENTOS A MAIOR'," + \
"                                            bruto STRING COMMENT 'VALOR BRUTO DA REMUNERAÇÃO'," + \
"                                            liquido STRING COMMENT 'VALOR LÍQUIDO APÓS DESCONTOS OBRIGATÓRIOS')" + \
"COMMENT 'Este conjunto de dados apresenta a remuneração dos servidores do Governo do DF, detalhada por órgão e nome do servidor.'"
)

Out[14]: DataFrame[]